# L2C

https://arxiv.org/abs/2209.13446

In [ ]:
#| default_exp methods.l2c

In [ ]:
#| include: false
%load_ext autoreload
%autoreload 2
from ipynb_path import *
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
#| export
from __future__ import annotations
from relax.import_essentials import *
from relax.methods.base import ParametricCFModule
from relax.base import BaseConfig
from relax.utils import *
from relax.data_utils import Feature, FeaturesList
from relax.ml_model import MLP, MLPBlock
from relax.data_module import DataModule
from keras_core.random import SeedGenerator
import einops

Using JAX backend.


An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [ ]:
#| hide
import torch
import relax

In [ ]:
#| export
def gumbel_softmax(
    key: jrand.PRNGKey, # Random key
    logits: Array, # Logits for each class. Shape (batch_size, num_classes)
    tau: float, # Temperature for the Gumbel softmax
):
    """The Gumbel softmax function."""

    gumbel_noise = jrand.gumbel(key, shape=logits.shape)
    y = logits + gumbel_noise
    return jax.nn.softmax(y / tau, axis=-1)

In [ ]:
#| export
def sample_categorical(
    key: jrand.PRNGKey, # Random key
    logits: Array, # Logits for each class. Shape (batch_size, num_classes)
    tau: float, # Temperature for the Gumbel softmax
    training: bool = True, # Apply gumbel softmax if training
):
    """Sample from a categorical distribution."""

    def sample_cat(key, logits):
        cat = jrand.categorical(key, logits=logits, axis=-1)
        return jax.nn.one_hot(cat, logits.shape[-1])

    return lax.cond(
        training,
        lambda _: gumbel_softmax(key, logits, tau=tau),
        lambda _: sample_cat(key, logits),
        None,
    )

In [ ]:
logits = jnp.array([[2.0, 1.0, 0.1], [1.0, 2.0, 3.0]])
key = jrand.PRNGKey(0)
output = sample_categorical(key, logits, tau=0.5, training=True)
assert output.shape == logits.shape
assert jnp.allclose(output.sum(axis=-1), 1.0)
# low temperature -> one-hot
output = sample_categorical(key, logits, tau=0.01, training=True)
assert jnp.array_equal(
    output.argmax(axis=-1), logits.argmax(axis=-1)
)
# high temperature -> uniform
output = sample_categorical(key, logits, tau=100, training=True)
assert jnp.max(output) - jnp.min(output) < 0.5

output = sample_categorical(key, logits, tau=0.5, training=False)
assert output.shape == logits.shape
assert jnp.array_equal(
    output.argmax(axis=-1), logits.argmax(axis=-1)
)

In [ ]:
#| export
def sample_bernouli(
    key: jrand.PRNGKey, # Random key
    prob: Array, # Logits for each class. Shape (batch_size, 1)
    tau: float, # Temperature for the Gumbel softmax
    training: bool = True, # Apply gumbel softmax if training
) -> Array:
    """"Sample from a bernouli distribution."""

    def sample_ber(key, prob):
        return jrand.bernoulli(key, p=prob).astype(prob.dtype)
    
    def gumbel_ber(key, prob, tau):
        key_1, key_2 = jrand.split(key)
        gumbel_1 = jrand.gumbel(key_1, shape=prob.shape)
        gumbel_2 = jrand.gumbel(key_2, shape=prob.shape)
        no_logits = (prob * jnp.exp(gumbel_1)) / tau
        de_logits = no_logits + ((1. - prob) * jnp.exp(gumbel_2)) / tau
        return no_logits / de_logits
    
    return lax.cond(
        training,
        lambda _: gumbel_ber(key, prob, tau),
        lambda _: sample_ber(key, prob),
        None,
    )

In [ ]:
#| export
class L2CModel(keras.Model):
    def __init__(
        self,
        generator_layers: list[int],
        selector_layers: list[int],
        feature_indices: list[tuple[int, int]] = None,
        pred_fn: Callable = None,
        alpha: float = 1e-4, # Sparsity regularization
        tau: float = 0.7,
        seed: int = None,
        **kwargs
    ):
        super().__init__(**kwargs)
        self.generator_layers = generator_layers
        self.selector_layers = selector_layers
        self.feature_indices = feature_indices
        self.pred_fn = pred_fn
        self.tau = tau
        self.alpha = alpha
        seed = seed or get_config().global_seed
        self.seed_generator = SeedGenerator(seed)

    def set_features_info(self, feature_indices: list[tuple[int, int]]):
        self.feature_indices = feature_indices
        # TODO: check if the feature indices are valid

    def set_pred_fn(self, pred_fn: Callable):
        self.pred_fn = pred_fn

    def build(self, input_shape):
        n_feats = len(self.feature_indices)
        self.generator = MLP(
            sizes=self.generator_layers,
            output_size=input_shape[-1],
            dropout_rate=0.0,
            last_activation="linear",
        )
        self.selector = MLP(
            sizes=self.selector_layers,
            output_size=n_feats,
            dropout_rate=0.0,
            last_activation="sigmoid",
        )

    def compute_l2c_loss(self, inputs, cfs, probs):
        y_target = self.pred_fn(inputs).argmin(axis=-1)
        y_pred = self.pred_fn(cfs)
        validity_loss = keras.losses.sparse_categorical_crossentropy(
            y_target, y_pred
        ).mean()
        sparsity = jnp.linalg.norm(probs, ord=1) * self.alpha
        # self.add_metric(validity_loss, name="validity_loss")
        # self.add_metric(sparsity, name="sparsity")
        return validity_loss, sparsity
    
    def perturb(self, inputs, cfs, probs, i, start, end):
        return cfs[:, start:end] * probs[:, i : i + 1] + inputs[:, start:end] * (1 - probs[:, i : i + 1])
    
    def forward(self, inputs, training=False):
        select_probs = self.selector(inputs, training=training)
        probs = sample_bernouli(
            self.seed_generator.next(), select_probs, 
            tau=self.tau, training=training
        )
        cfs_logits = self.generator(inputs, training=training)
        cfs = sample_categorical(
            self.seed_generator.next(), cfs_logits, 
            tau=self.tau, training=training
        )
        cfs = jnp.concatenate([
                self.perturb(inputs, cfs, probs, i, start, end)
                for i, (start, end) in enumerate(self.feature_indices)
            ], axis=-1,
        )
        return cfs, probs
    
    def call(self, inputs, training=False):
        cfs, probs = self.forward(inputs, training=training)
        # loss = self.compute_l2c_loss(inputs, cfs, probs)
        validity_loss, sparsity = self.compute_l2c_loss(inputs, cfs, probs)
        self.add_loss(validity_loss)
        self.add_loss(sparsity)
        return cfs   


In [ ]:
#| export
def qcut(
    x: Array, # Input array
    q: int, # Number of quantiles
    axis: int = 0, # Axis to quantile
) -> tuple[Array, Array]: # (digitized array, quantiles)
    """Quantile binning."""
    
    # Handle edge cases: empty array or single element
    if x.size <= 1:
        return jnp.zeros_like(x), jnp.array([])
    quantiles = jnp.quantile(x, jnp.linspace(0, 1, q + 1)[1:-1], axis=axis)
    
    digitized = jnp.digitize(x, quantiles)
    ohe_digitized = jax.nn.one_hot(digitized, q)
    quantiles = jnp.concatenate([
        x.min(axis=axis, keepdims=True), quantiles, 
        x.max(axis=axis, keepdims=True)])
    quantiles = (quantiles[1:] + quantiles[:-1]) / 2
    return ohe_digitized, quantiles

In [ ]:
digitized, quantiles = qcut(jnp.arange(10), 4)
assert digitized.shape == (10, 4)
assert quantiles.shape == (4,)
assert jnp.allclose(
    digitized, jax.nn.one_hot(jnp.array([0,0,0,1,1,2,2,3,3,3]), 4)
)

quantiles_true = jnp.array([0, 2.25, 4.5, 6.75, 9])
assert jnp.allclose(
    quantiles, (quantiles_true[1:] + quantiles_true[:-1]) / 2
)
x_empty = jnp.array([])
q = 2
digitized_empty, quantiles_empty = qcut(x_empty, q)
assert digitized_empty.size == 0 and quantiles_empty.size == 0
# Test with single element array
x_single = jnp.array([1])
digitized_single, quantiles_single = qcut(x_single, q)
assert digitized_single.size == 1 and quantiles_single.size == 0

# Test with large q value
xs = jnp.array([1, 2, 3, 4, 5, 6])
q_large = 10
_, quantiles_large = qcut(xs, q_large)
assert len(quantiles_large) == q_large

In [ ]:
#| export
def qcut_inverse(
    digitized: Array, # Digitized array
    quantiles: Array, # Quantiles
) -> Array:
    """Inverse of qcut."""
    
    return digitized @ quantiles

In [ ]:
digitized, quantiles = qcut(jnp.arange(10), 4)
quantiles_inv = qcut_inverse(digitized, quantiles)
assert quantiles_inv.shape == (10,)

In [ ]:
#| export
def discretize_xs(
    xs: Array, # Input array
    features_and_indices: list[tuple[Feature, tuple[int, int]]], # Features list
    q: int = 4, # Number of quantiles
) -> tuple[Array, list[tuple[tuple[int, int], Array]]]: # (discretized array, feature_indices_and_quantiles)
    """Discretize continuous features."""
    
    discretized_xs = []
    feature_indices_and_quantiles = []
    discretized_start, discretized_end = 0, 0

    for feat, (start, end) in features_and_indices:
        if feat.is_categorical:
            discretized = xs[:, start:end]
            quantiles = None
            discretized_end += end - start
        else:
            discretized, quantiles = qcut(xs[:, start:end].reshape(-1), q=q)
            discretized_end += discretized.shape[-1]      
        
        discretized_xs.append(discretized)
        feature_indices_and_quantiles.append(
            ((discretized_start, discretized_end), quantiles)
        )
        discretized_start = discretized_end
    discretized_xs = jnp.concatenate(discretized_xs, axis=-1)
    return discretized_xs, feature_indices_and_quantiles

In [ ]:
#| export
def inverse_discretize_xs(
    xs: Array, # Discretized input array
    feature_indices_and_quantiles: list[tuple[tuple[int, int], Array]], # Feature indices and quantiles
):
    """Continutize discretized features."""
    
    continutized_xs = []
    for (start, end), quantiles in feature_indices_and_quantiles:
        if quantiles is None:
            cont_feat = xs[:, start:end]
        else:
            cont_feat = qcut_inverse(xs[:, start:end], quantiles).reshape(-1, 1)
        continutized_xs.append(cont_feat)
            
    return jnp.concatenate(continutized_xs, axis=-1)

In [ ]:
#| export
def discretized_pred_fn(
    pred_fn: Callable, # Prediction function
    feature_indices_and_quantiles: list[tuple[tuple[int, int], Array]], # Feature indices and quantiles
) -> Callable[[Array], Array]:
    """Continutize discretized features."""
    
    def _pred_fn(xs):
        continutized_xs = inverse_discretize_xs(xs, feature_indices_and_quantiles)
        return pred_fn(continutized_xs)
    return _pred_fn

In [ ]:
#| export
class L2CConfig(BaseConfig):
    generator_layers: list[int] = Field(
        [64, 64, 64], description="Generator MLP layers."
    )
    selector_layers: list[int] = Field(
        [64], description="Selector MLP layers."
    )
    lr: float = Field(1e-3, description="Model learning rate.")
    opt_name: str = Field("adam", description="Optimizer name of training L2C.")
    alpha: float = Field(1e-4, description="Sparsity regularization.")
    tau: float = Field(0.7, description="Temperature for the Gumbel softmax.")
    q: int = Field(4, description="Number of quantiles.")

In [ ]:
#| export
class L2C(ParametricCFModule):
    def __init__(
        self,
        config: Dict | L2CConfig = None,
        l2c_model: L2CModel = None,
        name: str = "l2c",
    ):
        if config is None:
            config = L2CConfig()
        config = validate_configs(config, L2CConfig)
        name = name or "l2c"
        self.l2c_model = l2c_model
        super().__init__(config=config, name=name)

    def train(
        self, 
        data: DataModule, 
        pred_fn: Callable,
        batch_size: int = 128,
        epochs: int = 10,
        **fit_kwargs
    ):
        if not isinstance(data, DataModule):
            raise ValueError(f"Only support `data` to be `DataModule`, "
                             f"got type=`{type(data).__name__}` instead.")
        
        xs_train, ys_train = data['train']
        discretized_xs_train, self.feature_indices_and_quantiles = discretize_xs(
            xs_train, data.features.features_and_indices, q=self.config.q
        )
        pred_fn = discretized_pred_fn(pred_fn, self.feature_indices_and_quantiles)
        features_indices = [indices for indices, _ in self.feature_indices_and_quantiles]
        
        self.l2c_model = L2CModel(
            generator_layers=self.config.generator_layers,
            selector_layers=self.config.selector_layers,
            feature_indices=features_indices,
            pred_fn=pred_fn,
            alpha=self.config.alpha,
            tau=self.config.tau,
        )
        self.l2c_model.compile(
            optimizer=keras.optimizers.get({
                    'class_name': self.config.opt_name, 
                    'config': {'learning_rate': self.config.lr}
                }),
            loss=None
        )
        self.l2c_model.fit(
            discretized_xs_train, ys_train,
            epochs=epochs,
            batch_size=batch_size,
            **fit_kwargs
        )
        self._is_trained = True
        return self

In [ ]:
dm = relax.load_data('dummy')
ml_module = relax.load_ml_module('dummy')
l2c = L2C()
l2c.train(dm, ml_module.pred_fn)

discretize_xs=(750, 40)
features_indices=[(0, 4), (4, 8), (8, 12), (12, 16), (16, 20), (20, 24), (24, 28), (28, 32), (32, 36), (36, 40)]
continutized_xs.shape:  (128, 10)
continutized_xs.shape:  (128, 10)
Epoch 1/10
continutized_xs.shape:  (128, 10)
continutized_xs.shape:  (128, 10)
1/6 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - loss: 1.9377continutized_xs.shape:  (110, 10)
continutized_xs.shape:  (110, 10)
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 364ms/step - loss: 1.9441
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.8550 
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.8019
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.4649
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.2174
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1445
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9285
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7578
Epoch 9/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6021
Epoch 10/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s

<__main__.L2C>

In [ ]:
# @dataclass
# class Discretizer:
#     q: int = 4 # Number of quantiles

#     def fit(self, xs: Array):
#         _, self.quantiles = jax.vmap(qcut, in_axes=(1, None))(xs, self.q)
#         self.quantiles = jnp.concatenate(
#             [xs.min(axis=0).reshape(-1, 1), self.quantiles], axis=-1
#         ) # (n_feats, q)
#         return self

#     def transform(self, xs: Array):
#         if not hasattr(self, "quantiles"):
#             raise ValueError("Call fit before transform")
#         digitized = jax.vmap(jnp.digitize, in_axes=(1, 0))(xs, self.quantiles[:, 1:]).T
#         return digitized

#     def fit_transform(self, xs: Array):
#         digitized, self.quantiles = jax.vmap(qcut, in_axes=(1, None))(xs, self.q)
#         self.quantiles = jnp.concatenate(
#             [xs.min(axis=0).reshape(-1, 1), self.quantiles], axis=-1
#         )
#         return digitized.T

#     def inverse_transform(self, xs: Array):
#         return self.quantiles[xs]

In [ ]:
# xs = jnp.arange(40).reshape(5, 8)
# discretizer = Discretizer(q=4)
# discretizer.fit(xs)
# assert discretizer.quantiles.shape == (8, 4)
# digitized = discretizer.transform(xs)
# assert digitized.shape == (5, 8)

# assert jnp.allclose(
#     digitized,
#     einops.repeat(jnp.array([0, 1, 2, 3, 3]), 'i -> j i', j=8).T
# )

# digitized_ft = discretizer.fit_transform(xs)
# assert jnp.allclose(
#     digitized, digitized_ft
# )
